In [29]:
import cv2  
import numpy as np  
#import pdb  
#pdb.set_trace()#turn on the pdb prompt  
  
#read image  
img = cv2.imread('7elevenLogo.jpg',cv2.IMREAD_COLOR)  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
cv2.imshow('origin',img); 
  
#SIFT  
detector = cv2.SIFT()  
keypoints = detector.detect(gray,None)  
img = cv2.drawKeypoints(gray,keypoints)  
# img = cv2.drawKeypoints(gray,keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  
# print type(keypoints[0])
cv2.imshow('test',img);  
cv2.waitKey(0)  
cv2.destroyAllWindows()  

In [9]:
#read image  
img = cv2.imread('simitsu2.jpg',cv2.IMREAD_COLOR)  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
cv2.imshow('origin',img); 
  
#SIFT  
detector = cv2.SIFT()  
keypoints = detector.detect(gray,None)  
# img = cv2.drawKeypoints(gray,keypoints)  
img = cv2.drawKeypoints(gray,keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  
cv2.imshow('test',img);  
cv2.waitKey(0)  
cv2.destroyAllWindows()  

# 畫出match的畫

In [4]:
import numpy as np
import cv2

def drawMatches(img1, kp1, img2, kp2, matches):
    """
    My own implementation of cv2.drawMatches as OpenCV 2.4.9
    does not have this function available but it's supported in
    OpenCV 3.0.0

    This function takes in two images with their associated 
    keypoints, as well as a list of DMatch data structure (matches) 
    that contains which keypoints matched in which images.

    An image will be produced where a montage is shown with
    the first image followed by the second image beside it.

    Keypoints are delineated with circles, while lines are connected
    between matching keypoints.

    img1,img2 - Grayscale images
    kp1,kp2 - Detected list of keypoints through any of the OpenCV keypoint 
              detection algorithms
    matches - A list of matches of corresponding keypoints through any
              OpenCV keypoint matching algorithm
    """

    # Create a new output image that concatenates the two images together
    # (a.k.a) a montage
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        # Draw a small circle at both co-ordinates
        # radius 4
        # colour blue
        # thickness = 1
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)   
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points
        # thickness = 1
        # colour blue
        cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)


    # Show the image
    cv2.imshow('Matched Features', out)
    cv2.waitKey(0)
    cv2.destroyWindow('Matched Features')

    # Also return the image if you'd like a copy
    return out

## 比對兩張圖像

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('simitsu1.jpg',0)          # queryImage
img2 = cv2.imread('simitsu2.jpg',0) # trainImage

# Initiate SIFT detector
orb = cv2.ORB()

# find the keypoints and descriptors with SIFT
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

In [8]:
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 10 matches.
img3 = drawMatches(img1,kp1,img2,kp2,matches[:10])

plt.imshow(img3),plt.show()

(<matplotlib.image.AxesImage at 0xe5f5780>, None)

# 利用sift進行比對

In [31]:
import cv2  
import scipy as sp  
  
img1 = cv2.imread('7elevenLogo.jpg',0) # queryImage  
img2 = cv2.imread('7elevenTest2.jpg',0) # trainImage  
  
# Initiate SIFT detector  
sift = cv2.SIFT()  
  
# find the keypoints and descriptors with SIFT  
kp1, des1 = sift.detectAndCompute(img1,None)  
kp2, des2 = sift.detectAndCompute(img2,None)  
  
# FLANN parameters  
FLANN_INDEX_KDTREE = 0  
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)  
search_params = dict(checks=50)   # or pass empty dictionary  
flann = cv2.FlannBasedMatcher(index_params,search_params)  
matches = flann.knnMatch(des1,des2,k=2)  
  
print 'matches...',len(matches)  
# Apply ratio test  
good = []  
for m,n in matches:  
    if m.distance < 0.75*n.distance:  
        good.append(m)  
print 'good',len(good)  
# #####################################  
# visualization  
h1, w1 = img1.shape[:2]  
h2, w2 = img2.shape[:2]  
view = sp.zeros((max(h1, h2), w1 + w2, 3), sp.uint8)  
view[:h1, :w1, 0] = img1  
view[:h2, w1:, 0] = img2  
view[:, :, 1] = view[:, :, 0]  
view[:, :, 2] = view[:, :, 0]  
  
for m in good:  
    # draw the keypoints  
    # print m.queryIdx, m.trainIdx, m.distance  
    color = tuple([sp.random.randint(0, 255) for _ in xrange(3)])  
    #print 'kp1,kp2',kp1,kp2  
    cv2.line(view, (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1])) , (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1])), color)  
  
cv2.imshow("view", view)  
cv2.waitKey()  

matches... 191
good 19


-1

# 利用scipy比對

In [30]:
#coding=utf-8 
import cv2 
import scipy as sp 
  
img1 = cv2.imread('7elevenLogo.jpg',0) # queryImage 
img2 = cv2.imread('E1.png',0) # trainImage 
  
# Initiate SIFT detector 
sift = cv2.SIFT() 
  
# find the keypoints and descriptors with SIFT 
kp1, des1 = sift.detectAndCompute(img1,None) 
kp2, des2 = sift.detectAndCompute(img2,None) 
  
# FLANN parameters 
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5) 
search_params = dict(checks=50)  # or pass empty dictionary 
flann = cv2.FlannBasedMatcher(index_params,search_params) 
matches = flann.knnMatch(des1,des2,k=2) 
  
print 'matches...',len(matches) 
# Apply ratio test 
good = [] 
for m,n in matches: 
    if m.distance < 0.75*n.distance: 
        good.append(m) 
print 'good',len(good) 
# ##################################### 
# visualization 
h1, w1 = img1.shape[:2] 
h2, w2 = img2.shape[:2] 
view = sp.zeros((max(h1, h2), w1 + w2, 3), sp.uint8) 
view[:h1, :w1, 0] = img1 
view[:h2, w1:, 0] = img2 
view[:, :, 1] = view[:, :, 0] 
view[:, :, 2] = view[:, :, 0] 
  
for m in good: 
    #draw the keypoints 
    # print m.queryIdx, m.trainIdx, m.distance 
    color = tuple([sp.random.randint(0, 255) for _ in xrange(3)]) 
    ##print 'kp1,kp2',kp1,kp2 
    cv2.line(view, (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1])) , (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1])), color) 
  
cv2.imshow("view", view) 
cv2.waitKey() 

matches... 191
good 11


-1

# 儲存點特徵準備訓練

In [ ]:
import cv2
import numpy as np
import cPickle

img = cv2.imread('car.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT()
kp = sift.detect(gray, None)
img = cv2.drawKeypoints(img, keypoints = kp)

index = []

for point in kp:
    temp = (point.pt, point.size, point.angle, point.response, point.octave, 
            point.class_id) 
    index.append(temp)

## Put the keypoints into a File

f = open("keypoints.txt", "w")
f.write(cPickle.dumps(index))
f.close()
cv2.imwrite('sift_keypoints.jpg', img)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()